<a href="https://colab.research.google.com/github/Hieubui1103/Construction_/blob/main/LLaVa_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flash_attn


In [ ]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    use_flash_attention_2=True
).to(0)
processor = AutoProcessor.from_pretrained(model_id)

# Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image")




In [ ]:
%cd /content
!mkdir temp_clone
%cd temp_clone

# Sparse-checkout from here
!git clone --filter=blob:none --no-checkout https://github.com/Hieubui1103/Construction_.git
%cd Construction_
!git sparse-checkout init --cone
!git sparse-checkout set Images
!git checkout main

# get back to the main directory
%cd ../../

In [ ]:
import os
import shutil
import re


# Path to the folder containing 1000 images
image_folder_action = "temp_clone/Construction_/Images/Action_cons_files"
output_folder_action = "images_action"

image_folder_emotion = "temp_clone/Construction_/Images/Emotion_cons_files"
output_folder_emotion = "images_emotion"

# Create output folders if they don't exist
os.makedirs(output_folder_action, exist_ok=True)
os.makedirs(output_folder_emotion, exist_ok=True)



In [ ]:
def create_batches(image_folder, output_folder):
    """
    Create 5 batches of images from the specified folder and copy them into separate folders.
    """
    # Create 5 batch folders
    batch_size = 200
    for batch_num in range(1, 6):
        batch_folder = os.path.join(output_folder, f"batch_{batch_num}")
        os.makedirs(batch_folder, exist_ok=True)

    # Get all image files and sort them numerically
    image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])

    def extract_number(filename):
        match = re.search(r'\d+', filename)
        return int(match.group()) if match else -1

    image_files.sort(key=extract_number)

    # Distribute images into batches
    for i, file in enumerate(image_files):
        batch_num = (i // batch_size) + 1
        if batch_num > 5:  # Ensure no more than 5 batches
            break
        source_path = os.path.join(image_folder, file)
        destination_path = os.path.join(output_folder, f"batch_{batch_num}", file)
        shutil.copy(source_path, destination_path)

    print("Images have been copied into 5 batches.")


# Create batches for action images
create_batches(image_folder_action, output_folder_action)

# Create batches for emotion images
create_batches(image_folder_emotion, output_folder_emotion)

In [ ]:
import os

# Define the directory
image_dir_batch1 = "images_action/batch_1"
image_dir_batch2 = "images_action/batch_2"
image_dir_batch3 = "images_action/batch_3"
image_dir_batch4 = "images_action/batch_4"
image_dir_batch5 = "images_action/batch_5"

emotion_dir_batch1 = "images_emotion/batch_1"
emotion_dir_batch2 = "images_emotion/batch_2"
emotion_dir_batch3 = "images_emotion/batch_3"
emotion_dir_batch4 = "images_emotion/batch_4"
emotion_dir_batch5 = "images_emotion/batch_5"
# Get all files in the directory and sort them
def get_sorted_files(image_dir):
  image_files = sorted(
      [os.path.join(image_dir, file) for file in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, file))],
      key=lambda x: int(''.join(filter(str.isdigit, os.path.basename(x)))) if any(c.isdigit() for c in os.path.basename(x)) else x
  )
  return image_files

print(get_sorted_files(image_dir_batch5))  # Prints a sorted list of image file paths


In [ ]:
import json
import re
# Define the JSON file name

for i in range(1, 6):
  json_filename_action = f"response_action_LLaVa_{i}.json"
  json_filename_emotion = f"response_emotion_LLaVa_{i}.json"
  # Create an empty JSON structure
  data = {}

  # Write the empty structure to the file
  with open(json_filename_action, "w") as file:
      json.dump(data, file, indent=4)
  with open(json_filename_emotion, "w") as file:
      json.dump(data, file, indent=4)

  print(f"{json_filename_action} created successfully!")
  print(f"{json_filename_emotion} created successfully!")


In [ ]:
def save_data(data_dict, data,image, json_filename):
    print(image)
    data_dict[image] = data
    with open(json_filename, "w") as file:
        json.dump(data_dict, file, indent=4)

    print(f"Response saved in {json_filename}")


In [ ]:
# action classes
jobs = [
        "Operating heavy machinery",
                    "Lifting materials",
                    "Measuring and marking surfaces",
                    "Mixing cement",
                    "Collaborating in teams",
                    "Using hand tools",
                    "Inspecting completed work",
                    "Communicating with supervisors",
                    "Following safety protocols",
                    "Taking breaks/resting"
        ]

# emotion classes
emotions = [
        "Focused",
        "Determined",
        "Tired",
        "Alert",
        "Satisfied",
        "Anxious",
        "Proud",
        "Frustrated",
        "Cooperative",
        "Relieved"
        ]

Answer retrieval with direct questioning

In [ ]:
def retrieve_ans(image_files, json_file, category, classes):
  data = {}
  for image in image_files:
    print(f"\n Current process image: {image}  \n")
    answer_data_for_each = []
    for job in classes:
      conversation = [
          {

            "role": "user",
            "content": [
                {"type": "text", "text": f""" Does this {category} of '{job}' appear in this image? Only answer yes or no"""},
                {"type": "image"},
              ],
          },
      ]
      prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

      raw_image = Image.open(image)
      inputs = processor(images=raw_image, text=prompt, return_tensors='pt').to(0, torch.float16)

      output = model.generate(**inputs, max_new_tokens=10000, do_sample=False)
      response_1 = processor.decode(output[0][2:], skip_special_tokens=True)

      match = re.search(r"ASSISTANT:\s*(\w+)", response_1)
      if match:
          answer = match.group(1)
          answer_data_for_each.append(answer)
      else:
          print("No match found.")
      print(response_1)

      # Clear cache
      del inputs, output, response_1, match
      torch.cuda.empty_cache()

    print(answer_data_for_each)
    save_data(data, answer_data_for_each,image, json_file)
    del answer_data_for_each
    torch.cuda.empty_cache()
  del data
  torch.cuda.empty_cache()

Get data for action recognition


In [ ]:
retrieve_ans(get_sorted_files(image_dir_batch1), "response_action_LLaVa_1.json", "action", jobs)

In [ ]:
retrieve_ans(get_sorted_files(image_dir_batch2), "response_action_LLaVa_2.json", "action", jobs)

In [ ]:
retrieve_ans(get_sorted_files(image_dir_batch3), "response_action_LLaVa_3.json", "action", jobs)

In [ ]:
retrieve_ans(get_sorted_files(image_dir_batch4), "response_action_LLaVa_4.json", "action", jobs)

In [ ]:
retrieve_ans(get_sorted_files(image_dir_batch5), "response_action_LLaVa_5.json", "action", jobs)

Get data for emotion recognition

In [ ]:
retrieve_ans(get_sorted_files(emotion_dir_batch1), "response_emotion_LLaVa_1.json", "emotion", emotions)

In [ ]:
retrieve_ans(get_sorted_files(emotion_dir_batch2), "response_emotion_LLaVa_2.json", "emotion", emotions)

In [ ]:
retrieve_ans(get_sorted_files(emotion_dir_batch3), "response_emotion_LLaVa_3.json", "emotion", emotions)

In [ ]:
retrieve_ans(get_sorted_files(emotion_dir_batch4), "response_emotion_LLaVa_4.json", "emotion", emotions)

In [ ]:
retrieve_ans(get_sorted_files(emotion_dir_batch5), "response_emotion_LLaVa_5.json", "emotion", emotions)